In [1]:
#IMPORTS
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
#MODIFIER DOMAIN VARIABLES

def mod_func_A(x):
    return x**2

def mod_func_B(x):
    return x*2/3

def rescaler(old_list, new_min, new_max):
    new_values=[]
    for old_value in old_list:
        new_values.append((((old_value - min(old_list)) * (new_max - new_min)) / (max(old_list) - min(old_list)))+ new_min)

    return new_values

# datapoints domain (min-max), min_mod_increment, max_mod_increment 
mdv={"domain_min_range":1, "domain_max_range":100, "modifier_incremental_unit":3, "modifier_data_point":10}
#mdv["modifier_data_point"] = 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt

def modifier_controller(interval_list=[(mdv["domain_min_range"], mdv["domain_max_range"])], selected_mod_function=mod_func_A, do_plot=False):
    # Function to control modifiers given the input and the selected modifier function. Option to plot or not. 
    
    print("\nModifier controller...")
    print('  * Interval: ',interval_list)
    all_interval_mod = []
    
    # Check if it's possible to generate more data points
    if mdv["modifier_data_point"] < mdv["modifier_incremental_unit"]:
        return False  # Exit the function if not possible
    
    for i in range(len(interval_list)):
        interval_min_range = interval_list[i][0]
        interval_max_range = interval_list[i][1]
        
        # Generate data points (incremental ticks and function modified x values) within the specified interval
        mod_ticks = np.arange(interval_min_range, interval_max_range, mdv["modifier_data_point"])
        mod_x = selected_mod_function(mod_ticks)
        
        # Normalize the function outputs to fit within the interval range
        mod_x = rescaler(mod_x, new_max=interval_max_range, new_min=interval_min_range)
        all_interval_mod.append(mod_x)
    
    # update the mdv to decrease the interdatapoint distance for the next itteration
    mdv["modifier_data_point"] = mdv["modifier_data_point"] - mdv["modifier_incremental_unit"]
    
    if do_plot == True:
        # Plot the generated data points
        for mod_x in all_interval_mod:
            plt.scatter(mod_x, np.ones(np.shape(mod_x)))
            plt.show()
    
    print('  * Mod_x:   ',all_interval_mod)

    return all_interval_mod


In [22]:
#SIMULATOR - data substitution in Sim function
def sim_func_A(x):
    noise = np.random.normal(-10000,10000,len(x))
    return x**3-x**2+noise

def sim_func_B(x):
    return x*2/3

def simulator(mod_x, selected_function=sim_func_A):
    print("\nSimulator...")
    all_interval_sim =[]
    if mod_x == False: 
        return False # possible itterations have ended
    else:
        for i in range(len(mod_x)):
            simulated_y = selected_function(np.array(mod_x[i]))
            all_interval_sim.append(list(simulated_y))
    print('  * Sim_y:   ',all_interval_sim)
    return all_interval_sim


## Part TWO
modifier and simulator working in dynamic way, with hardcoded ranges
* next step, extract the ranges 



In [29]:
# PLACEHOLDER FOR THE MAIN FUNCTION V2
from Validator import Validator

# Instantiate mdv values
mdv={"domain_min_range":1, "domain_max_range":100, "modifier_incremental_unit":3, "modifier_data_point":10}
# Create an instance of the Validator Class
validate = Validator()
appendedvars=[]

mainfunc=True
# Initialize interval list
interval_lists=[(mdv["domain_min_range"], mdv["domain_max_range"])]
x=1
while mainfunc==True:
    # Run Modifier Controller Function
    mod_x_list = modifier_controller(interval_list=interval_lists,do_plot=False,selected_mod_function=mod_func_A)
    # appendedvars.append(mod_x_list)
    if mod_x_list == False:
        print('*   ITTERATIONS END HERE   *')

    # Run Simulator Fuction
    # for i in range(np.shape(mod_x_list)[0]):
    sim_y_list = simulator(mod_x_list,selected_function=sim_func_A)
    assert len(mod_x_list) == len(sim_y_list)
    # Get ranges of unfit points ( IF ANY )
    # ranges = validate.get_unfitting_ranges(mod_x_list,sim_y_list)
    if x == 1: 
        ranges = [(0,50),(70,80)]
        x=2
    elif x == 2:
        ranges = [(30,50),(70,75)]
        x=3
    elif x==3: 
        ranges = False
    if mod_x_list == False or not ranges: 
        mainfunc=False
        print('The END')
    interval_lists = ranges


# TODO: 
# Validator needs to work as a loop within itseft that accepts 3 different sets of points
# understand curve fitting
# Fix ranges



Modifier controller...
  * Interval:  [(1, 100)]
  * Mod_x:    [[1.0, 2.4347826086956523, 6.260869565217392, 12.478260869565217, 21.08695652173913, 32.086956521739125, 45.47826086956522, 61.26086956521739, 79.43478260869566, 100.0]]

Simulator...
  * Sim_y:    [[-11273.107964648845, 8142.890776677715, 6174.731463487686, -2019.8458862030673, -5917.804000472208, 25718.744267449438, 104343.32127167613, 198971.1534319353, 483662.49297055294, 973739.9213716126]]

Modifier controller...
  * Interval:  [(0, 50), (70, 80)]
  * Mod_x:    [[0.0, 1.0204081632653061, 4.081632653061225, 9.183673469387756, 16.3265306122449, 25.510204081632654, 36.734693877551024, 50.0], [70.0, 80.0]]

Simulator...
  * Sim_y:    [[-16511.73982812568, -22377.29396016328, -7950.980697156935, -2893.308365052671, -21078.75139600024, 24005.615157655324, 27299.275611881927, 98997.77162918734], [329039.11028990167, 503330.55385030725]]

Modifier controller...
  * Interval:  [(30, 50), (70, 75)]
  * Mod_x:    [[30.0, 34.210

In [30]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.linear_model import HuberRegressor
# from sklearn.preprocessing import StandardScaler

# x_values = mod_x_list
# y_values = sim_y_list

# x_scaler, y_scaler = StandardScaler(), StandardScaler()
# x_train = x_scaler.fit_transform(x_values)
# y_train = y_scaler.fit_transform(y_values)

# # fit model
# model = HuberRegressor(epsilon=1)
# model.fit(x_train, y_train.ravel())

# # Fit curve to the data
# fitted_curve = np.polyfit(x_values, y_values, 1)
# return fitted_curve